In [1]:
from generator import *
from search import *

print(content_generator("!generate pato"))

2023-05-21 19:47:24.655907: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-21 19:47:24.695389: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-21 19:47:24.696152: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-21 19:47:25.447930: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [1]:
command = "!search pato"

In [2]:
import re

pattern = r"!search (.+)(?:\sth=(\d+(\.\d+)?))?"
match = re.match(pattern, command)

if match:
    term = match.group(1)
    threshold = match.group(2)
        
    if threshold is not None:
        threshold = float(threshold)

# else:
#     match = re.match(r"!search (.+)", command)

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd


In [4]:

def content_filter(content):

    bad_words = 'datasets/bad_words.csv'
    good_words = 'datasets/words_pos.csv'

    bad_words = pd.read_csv(bad_words)
    good_words = pd.read_csv(good_words)


    good_words = good_words.drop(columns=['pos_tag'])
    good_words['IsBad'] = 0
    bad_words['IsBad'] = 1

    good_words_sample = good_words.sample(1618, random_state=42)

    words = pd.concat([good_words_sample, bad_words])

    X = words["word"]
    y = words["IsBad"]

    X_train, X_test, y_train, y_test = train_test_split(X, y ,test_size=0.2, random_state=42)

    classificador = Pipeline([
                        ('meu_vetorizador', CountVectorizer(stop_words='english')),
                        ('meu_classificador', LogisticRegression(penalty=None, solver='saga', max_iter=10000))
                        ])
    
    classificador.fit(X_train,y_train)
    y_pred = classificador.predict(X_test)
    acc = accuracy_score(y_pred,y_test)
    print(acc)

    prob = classificador.predict_log_proba([content])
    probas = classificador.predict_proba(X_train)

    m = np.max(probas)
    prob = 2 * (m - prob) / (2 * m) - 1
    return prob[0][1]


In [5]:
th = content_filter("ass")

if th > 0.5:
    print("yup")

0.45904173106646057


In [6]:

bad_words = 'datasets/bad_words.csv'
good_words = 'datasets/words_pos.csv'

bad_words = pd.read_csv(bad_words)
good_words = pd.read_csv(good_words)


good_words = good_words.drop(columns=['pos_tag'])
good_words['IsBad'] = 0
bad_words['IsBad'] = 1

good_words_sample = good_words.sample(1618, random_state=42)

words = pd.concat([good_words_sample, bad_words])

In [7]:
words

,word,IsBad
219193,overemployment,0
119423,frostflower,0
251834,progeneration,0
319490,terracer,0
258313,quemeful,0
...,...,...
1612,cocky,1
1613,transsexual,1
1614,unfuckable,1
1615,bestiality,1
